#### Assignment 2 - Alex Ceccotti - 790497

In [1]:
import os
import cv2 as cv
from matplotlib import pyplot as plt
from skimage import feature
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from time import time

In [2]:
def load_data(feature_extractor, maximages=50, base_path='./classes/', extension='.jpg'):
    labels = []
    features = []
    for di,d in enumerate(sorted(os.listdir(base_path))):
        for fi,f in enumerate(sorted(os.listdir(base_path + d + '/'))):
            if f.endswith(extension) and fi<maximages:
                image = cv.imread(base_path + d + '/' + f, 0)
                cur_features = feature_extractor(image)
                features.append(cur_features)
                labels.append(di)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, shuffle=True, random_state=1)
    return X_train, X_test, y_train, y_test

def lbp_features(img, P=24, R=8, shift=8, dims=16):
    img = feature.local_binary_pattern(img, P=P, R=R, method='uniform')
    feats = []
    for i in range(0,img.shape[0],shift):
        for j in range(0,img.shape[1],shift):
            hist = np.bincount(img[i:i+dims,j:j+dims].flatten().astype(int), minlength=P+2)
            feats.append(hist)
    return np.array(feats)

def bow_features(feats, dictionary, nwords):
    quantized = dictionary.predict(feats)
    t = np.bincount(quantized, minlength=nwords)
    return t

def bow(X_train, X_test, nwords, normalize=True, eps=0.001):
    X_train_stack = np.zeros((0,X_train[0].shape[1]), dtype=np.float32)
    for t in X_train:
        X_train_stack = np.concatenate((X_train_stack, t))
    if normalize:
        X_train_mean = X_train_stack.mean(axis=0)
        X_train_std = X_train_stack.std(axis=0)
        X_train = [(t - X_train_mean + eps)/(X_train_std + eps) for t in X_train]
        X_test = [(t - X_train_mean + eps)/(X_train_std + eps) for t in X_test]
        X_train_stack = (X_train_stack - X_train_mean + eps)/(X_train_std + eps)
    dictionary = MiniBatchKMeans(n_clusters=nwords)
    dictionary.fit(X_train_stack)
    X_train = [bow_features(f, dictionary, nwords) for f in X_train]
    X_test = [bow_features(f, dictionary, nwords) for f in X_test]
    X_train = [hist/hist.sum() for hist in X_train]
    X_test = [hist/hist.sum() for hist in X_test]
    return X_train, X_test

La funzione "load_data" consente di caricare delle immagini a partire da diverse cartelle, ognuna delle quali contiene immagini con una determinata label associata. Durante il caricamento delle immagini, vengono in realtà estrapolate delle feature tramite la funzione "lbp_feature". Viene quindi utilizzato lbp su tutta l'immagine e viene fatta scorrere una finestra. Ad ogni scorrimento si calcola l'istogramma della finestra che rappresenta una feature dell'immagine. La funzione "bow" crea un "dizionario" realizzato tramite k-means a partire dalle features normalizzate. Successivamente, tramite la funzione "bow_feature", ogni feature di ogni immagine viene associata ad uno dei centroidi precedentemente trovati. Le feature relative ad un'immagine diventano dunque l'istogramma normalizzato delle "bag of words" associate a quell'immagine.

In [3]:
X_train, X_test, y_train, y_test = load_data(feature_extractor=lambda x:
                                                    lbp_features(x, P=24, R=8, shift=8, dims=16), maximages=500)

Si estraggono le feature di partenza utilizzando LBP con 24 punti e raggio 8. La finestra mobile è di dimensioni 16x16 e viene fatta scorrere di 8 pixel alla volta, sia in orizzontale sia in verticale.

In [4]:
X_train2, X_test2 = bow(X_train, X_test, 300)

Viene trovato il dizionario (con 300 "parole") e si trovano le feature finali per ogni immagine. Ogni immagine avrà dunque 300 feature, ognuna delle quali indica la frequenza con cui una data "parola" viene associata all'immagine.

In [5]:
#param_grid = {'C': [1, 5, 10, 50, 100, 500],
#          'gamma': [1, 5, 10, 50, 100, 500], } 
#C=1, gamma=500

param_grid = {'C': [0.5, 1, 1.5, 2],
          'gamma': [100, 200, 300, 400, 500, 600], } 
#C=1.5, gamma=300

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=5, n_jobs=-1)

t2 = time()
clf = clf.fit(X_train2, y_train)
print("Addestramento completato in %0.3fs" % (time() - t2))

print("Migliore combinazione di parametri:")
print(" C: "+str(clf.best_estimator_.C))
print(" gamma: "+str(clf.best_estimator_.gamma))

C:\Users\alexc\Anaconda3\envs\dsim\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Addestramento completato in 35.259s
Migliore combinazione di parametri:
 C: 1.5
 gamma: 300


Viene allenata una SVM con Cross Validation (5 folds) a partire da una griglia generica. Una volta trovati dei parametri idonei, viene fatta una ricerca più centrata sui valori così ottenuti (C=1.5 e gamma=300). I parametri finali ottenuti con il secondo training sono C=1.5 e gamma=300.

In [6]:
y_pred = clf.predict(X_test2)

print("Report di classificazione:")
print(classification_report(y_test, y_pred))

Report di classificazione:
              precision    recall  f1-score   support

           0       0.71      0.75      0.73       149
           1       0.74      0.70      0.72       151

   micro avg       0.73      0.73      0.73       300
   macro avg       0.73      0.73      0.73       300
weighted avg       0.73      0.73      0.73       300



I risulti sul test set sono quelli riportati nella tabella soprastante. In particolare, si ottiene un f1-score pari a 0.73.

In [7]:
print("Matrice di confusione:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("Accuracy: " + str(accuracy_score(y_test, y_pred)))

Matrice di confusione:
[[112  37]
 [ 45 106]]
Accuracy: 0.727


Qui sopra viene riportata la matrice di confusione e si evidenzia il risultato in termini di accuracy: 0.727.